### 패키지 임포트

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import copy
from collections import OrderedDict

### 그림 그리는 함수

In [2]:
def show_v_table_small(v_table, env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1])
        print("|", end="")
        for j in range(env.reward.shape[1]):
            print("{0:8.2f}  |".format(v_table[i,j]),end="")
        print()
    print("+----------"*env.reward.shape[1])

# V table 그리기    
def show_v_table(v_table, env):    
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                        print("   {0:8.2f}      |".format(v_table[i,j]),end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    
# Q table 그리기
def show_q_table(q_table,env):
    for i in range(env.reward.shape[0]):
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("{0:10.2f}       |".format(q_table[i,j,0]),end="")
                if k==1:
                    print("{0:6.2f}    {1:6.2f} |".format(q_table[i,j,3],q_table[i,j,1]),end="")
                if k==2:
                    print("{0:10.2f}       |".format(q_table[i,j,2]),end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    

# 정책 policy 화살표로 그리기
def show_q_table_arrow(q_table,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    if np.max(q_table[i,j,:]) == q_table[i,j,0]:
                        print("        ↑       |",end="")
                    else:
                        print("                 |",end="")
                if k==1:                    
                    if np.max(q_table[i,j,:]) == q_table[i,j,1] and np.max(q_table[i,j,:]) == q_table[i,j,3]:
                        print("      ←  →     |",end="")
                    elif np.max(q_table[i,j,:]) == q_table[i,j,1]:
                        print("          →     |",end="")
                    elif np.max(q_table[i,j,:]) == q_table[i,j,3]:
                        print("      ←         |",end="")
                    else:
                        print("                 |",end="")
                if k==2:
                    if np.max(q_table[i,j,:]) == q_table[i,j,2]:
                        print("        ↓       |",end="")
                    else:
                        print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")    
    
# 정책 policy 화살표로 그리기
def show_policy_small(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1],end="")
        print("+")
        print("|", end="")
        for j in range(env.reward.shape[1]):
            if env.reward_list1[i][j] == "road":
                if policy[i,j] == 0:
                    print("   ↑     |",end="")
                elif policy[i,j] == 1:
                    print("   →     |",end="")
                elif policy[i,j] == 2:
                    print("   ↓     |",end="")
                elif policy[i,j] == 3:
                    print("   ←     |",end="")
            else:
                print("          |",end="")
        print()
    print("+----------"*env.reward.shape[1],end="")
    print("+")
    
# 정책 policy 화살표로 그리기
def show_policy(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                    if policy[i,j] == 0:
                        print("      ↑         |",end="")
                    elif policy[i,j] == 1:
                        print("      →         |",end="")
                    elif policy[i,j] == 2:
                        print("      ↓         |",end="")
                    elif policy[i,j] == 3:
                        print("      ←         |",end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")

### Agent 구현

In [3]:
class Agent():
    
    # 1. 행동에 따른 에이전트의 좌표 이동(위, 오른쪽, 아래, 왼쪽) 
    action = np.array([[-1,0],[0,1],[1,0],[0,-1]])
    
    # 2. 각 행동별 선택확률
    select_action_pr = np.array([0.25,0.25,0.25,0.25])
    
    # 3. 에이전트의 초기 위치 저장
    def __init__(self):
        self.pos = (0,0)
    
    # 4. 에이전트의 위치 저장
    def set_pos(self,position):
        self.pos = position
        return self.pos
    
    # 5. 에이전트의 위치 불러오기
    def get_pos(self):
        return self.pos

### Environment 구현

# 수정사항: 도착하는 순간 done = False -> done = True

In [4]:
class Environment():
    
    # 1. 미로밖(절벽), 길, 목적지와 보상 설정
    cliff = -3
    road = -1
    goal = 1
    
    # 2. 목적지 좌표 설정
    goal_position = [2,2]
    
    # 3. 보상 리스트 숫자
    reward_list = [[road,road,road],
                   [road,road,road],
                   [road,road,goal]]
    
    # 4. 보상 리스트 문자
    reward_list1 = [["road","road","road"],
                    ["road","road","road"],
                    ["road","road","goal"]]
    
    # 5. 보상 리스트를 array로 설정
    def __init__(self):
        self.reward = np.asarray(self.reward_list)    

    # 6. 선택된 에이전트의 행동 결과 반환 (미로밖일 경우 이전 좌표로 다시 복귀)
    def move(self, agent, action):
        
        done = False
        
        # 6.1 현재좌표가 목적지인지 확인
        if self.reward_list1[agent.pos[0]][agent.pos[1]] == "goal":
            reward = self.goal
            observation = agent.set_pos(agent.pos)
            done = True
            return observation, reward, done
        
        
        # 6.2 행동에 따른 좌표 구하기
        new_pos = agent.pos + agent.action[action]
        
        
        
        # 6.3 이동 후 좌표가 미로 밖인지 확인    
        if new_pos[0] < 0 or new_pos[0] >= self.reward.shape[0] or new_pos[1] < 0 or new_pos[1] >= self.reward.shape[1]:
            reward = self.cliff
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.3 이동 후 좌표가 목적지인지 확인
        elif self.reward_list1[new_pos[0]][new_pos[1]] == "goal":
            reward = self.goal
            observation = agent.set_pos(new_pos)
            done = True
        # 6.4 이동 후 좌표가 길이라면
        else:
            observation = agent.set_pos(new_pos)
            reward = self.reward[observation[0],observation[1]]
            
        return observation, reward, done

### 함수 근사화를 이용하여 행동가치함수 학습하기

- 간단한 선형 함수를 이용하여 Q Table을 근사화한다. 
- Stochastic Gradient Descent를 이용하여 근사 함수를 학습한다

In [5]:

def find_and_show_q_table(Q, env, agent):
    for k in range(4):
        print("Q[i, j, %c ] = %.2f×i%+.2f×j%+.2f"%("상우하좌"[k], a1[k], a2[k], a3[k]))
    q_table = np.zeros((env.reward.shape[0], env.reward.shape[1], len(agent.action)))
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            for k in range(len(agent.action)): 
                q_table[i, j, k] = Q(i, j, k)
    print("Q Table (approximated by linear function)")
    show_q_table(np.round(q_table,2),env)
    print("Best actions Arrow")
    show_q_table_arrow(q_table,env)

np.random.seed(7777)

# 1. 환경 초기화
env = Environment()

# 2. 에이전트 초기화
agent = Agent()


# 3 행동의 가치를 함수 형식으로 저장
a1 = np.random.normal(scale=0.1,size=[len(agent.action)])
a2 = np.random.normal(scale=0.1,size=[len(agent.action)])
a3 = np.random.normal(scale=0.1,size=[len(agent.action)])
Q = lambda i, j, act: a1[act]*i + a2[act]*j + a3[act]

total_step = 2000
batch_size = 100
steps_for_print = 400
gamma = 0.9
learning_rate = 0.01
total_loss = 0

print("="*30)
print("step", 0)
find_and_show_q_table(Q, env, agent)

for attmp in range(total_step):
    #기울기 저장
    a1_gradient = np.array([0, 0, 0, 0], dtype=float)
    a2_gradient = np.array([0, 0, 0, 0], dtype=float)
    a3_gradient = np.array([0, 0, 0, 0], dtype=float)
    
    for _ in range(batch_size):
        i, j = np.random.randint(0, 3, [2])
        while (i, j) == (2, 2):
            i, j = np.random.randint(0, 3, [2])
        agent.set_pos([i,j])
        act = np.random.randint(len(agent.action))
        observation, reward, done = env.move(agent, act)
        if not done:
            next_q = -1e6
            # Q-Learning Style
            for k in range(len(agent.action)):
                next_q = max( Q(observation[0], observation[1], k), next_q)
            target = reward + gamma * next_q
        else:
            target = reward
        # 오차 구하기
        # (Mean) Square Error, MSE
        # MSE = 1/2 Σ(Pred - Target) ^ 2 
        loss = 1/2 * (Q(i, j, act) - target)**2
        total_loss += loss
        
        # ∂Loss /∂Q
        Q_gradient = Q(i, j, act) - target
        
        # ∂Loss/∂a = (∂Loss/∂Q) × (∂Q/∂a)
        # Q = a1 × i + a2 × j + a3
        # (∂Q/∂a1) = i, (∂Q/∂a2) = j, (∂Q/∂a3) = 1    
        
        #∂Loss/∂a1 = (∂Loss/∂Q) × (∂Q/∂a1) = Q_gradient × i 
        a1_gradient[act] += Q_gradient * i
        #∂Loss/∂a2 = (∂Loss/∂Q) × (∂Q/∂a2) = Q_gradient × j
        a2_gradient[act] += Q_gradient * j
        #∂Loss/∂a3 = (∂Loss/∂Q) × (∂Q/∂a1) = Q_gradient × 1
        a3_gradient[act] += Q_gradient
    
    # 평균 (mean)
    a1_gradient /= batch_size
    a2_gradient /= batch_size
    a3_gradient /= batch_size
    
    # 파라미터 갱신 (훈련)
    a1 -= learning_rate*a1_gradient
    a2 -= learning_rate*a2_gradient
    a3 -= learning_rate*a3_gradient
      
    # 주기적으로 평균 Loss 출력
    if (attmp+1) % steps_for_print == 0:
        print("="*30)
        print("step", attmp+1, " loss:", total_loss/(steps_for_print*batch_size))
        total_loss = 0
        find_and_show_q_table(Q, env, agent)
        
#4. 학습 후 차이(편차) 계산
devi_table = np.zeros((env.reward.shape[0], env.reward.shape[1],len(agent.action)))

for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        if (i, j) == (2, 2):
            continue
        for k in range(4):
            agent.set_pos([i,j])
            observation, reward, done = env.move(agent, k)
            if not done:
                next_q = -1e6
                # Q-Learning Style
                for inner_k in range(len(agent.action)):
                    next_q = max( Q(observation[0], observation[1], inner_k), next_q)
                target = reward + gamma * next_q
            else:
                target = reward
            devi_table[i,j,k] = Q(i,j,k) - target
            
print()
print("Error Table")
show_q_table(np.round(devi_table,2), env)


step 0
Q[i, j, 상 ] = 0.06×i+0.28×j+0.01
Q[i, j, 우 ] = 0.10×i+0.10×j+0.04
Q[i, j, 하 ] = 0.01×i-0.03×j-0.05
Q[i, j, 좌 ] = -0.00×i+0.05×j-0.06
Q Table (approximated by linear function)
+-----------------+-----------------+-----------------+
|      0.01       |      0.29       |      0.57       |
| -0.06      0.04 | -0.01      0.14 |  0.04      0.24 |
|     -0.05       |     -0.08       |     -0.11       |
+-----------------+-----------------+-----------------+
|      0.07       |      0.35       |      0.63       |
| -0.06      0.13 | -0.01      0.24 |  0.04      0.34 |
|     -0.04       |     -0.07       |     -0.10       |
+-----------------+-----------------+-----------------+
|      0.13       |      0.41       |      0.69       |
| -0.07      0.23 | -0.01      0.33 |  0.04      0.43 |
|     -0.03       |     -0.06       |     -0.09       |
+-----------------+-----------------+-----------------+
Best actions Arrow
+-----------------+-----------------+-----------------+
|              

step 2000  loss: 0.46277460081359734
Q[i, j, 상 ] = -0.21×i-0.40×j-2.03
Q[i, j, 우 ] = 0.25×i-0.40×j-1.96
Q[i, j, 하 ] = -0.39×i+0.26×j-1.93
Q[i, j, 좌 ] = -0.41×i-0.22×j-2.03
Q Table (approximated by linear function)
+-----------------+-----------------+-----------------+
|     -2.03       |     -2.44       |     -2.84       |
| -2.03     -1.96 | -2.24     -2.36 | -2.46     -2.75 |
|     -1.93       |     -1.67       |     -1.41       |
+-----------------+-----------------+-----------------+
|     -2.25       |     -2.65       |     -3.05       |
| -2.43     -1.70 | -2.65     -2.10 | -2.87     -2.50 |
|     -2.32       |     -2.06       |     -1.80       |
+-----------------+-----------------+-----------------+
|     -2.46       |     -2.87       |     -3.27       |
| -2.84     -1.45 | -3.05     -1.85 | -3.27     -2.24 |
|     -2.71       |     -2.45       |     -2.19       |
+-----------------+-----------------+-----------------+
Best actions Arrow
+-----------------+-----------------+--

### 신경망 기초

#### 이후 코드는 "밑바닥부터 시작하는 딥러닝"의 Github 코드를 가져오고 수정하여 작성한 코드임

- 신경망의 층들은 기본적으로 연산하는 forward와 출력에 대한 입력 및 파라미터들의 편미분을 구하는 backward로 나누어져 있다. 이러한 방식을 backpropagation이라고 한다.
- forward에서 입력된 입력들을 기억해놓았다가 backward에서 gradient를 구하는데 사용된다.
- 결과 값에 대하여 관여한 모든 파라미터들의 Gradient 값을 빠르게 구할 수 있게 된다.

### 행렬곱 (Affine, 어파인) 층

In [6]:
# 함수를 이루는 신경망의 파라미터들이 어떻게 학습되는지만 이해해보도록 합시다.

class Affine:
    def __init__(self, W, b):
        # (D, H)
        self.W = W
        # (H)
        self.b = b
        
        #forward에서 입력된 값을 기억하고 있다가
        #backward에서 gradient를 구할 때 사용된다.
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    # y = x W + b
    def forward(self, x):
        # (다차원) 텐서 대응
        self.original_x_shape = x.shape
        #x의 첫번째 차원은 batch의 차원에 해당하는 부분이다.
        x = x.reshape(x.shape[0], -1)
        self.x = x

        # (B, H)   (B, D) (D, H)        (H) : broadcasting (B, H)
        #   y    =   x      W       +    b
        out = np.dot(self.x, self.W) + self.b

        return out

    # ∂L/∂W = x.T (∂L/∂y)
    # ∂L/∂b = (∂L/∂y) (broadcasting에 의한 것만 덧셈해주면 됨)
    # ∂L/∂x = (∂L/∂y) W.T
    def backward(self, dout):
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        dx = np.dot(dout, self.W.T)
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)

        return dx

In [7]:
# 차원 설정
# 배치 개수
batch_size = 2
# 입력 차원
D = 4
# Affine 출력 차원
H = 3

# 입력 및 파라미터 선언
x = np.random.normal(size=[batch_size, D])
W = np.random.normal(size=[D, H])
b = np.random.normal(size=[H])

# 어파인 층 선언
A_layer = Affine(W, b)

# 순전파
y = A_layer.forward(x)

print("xW + b = ")
print(y)

# 원래대로라면, 최종 출력으로 부터 이어져 온 y의 편미분 그래디언트의 자리이다
# 여기서는 모든 원소가 1인 행렬이라고 가정하자
dout = np.ones_like(y)

#역전파
dx = A_layer.backward(dout)

print()
print("dx = ")
print(dx)

print("x의 [0, 0]과 곱해진 W 원소들의 합")
print(np.sum(W[0, :]))

print()
print("dw = ")
print(A_layer.dW)

print("W의 [0, 0]과 곱해진 x 원소들의 합")
print(np.sum(x[:, 0]))

print()
print("db = ")
print(A_layer.db)


xW + b = 
[[ 0.59798385 -4.79014224  2.4224355 ]
 [ 2.70411312 -4.489438    1.89747805]]

dx = 
[[-3.07697927  2.50687006 -0.69469937  0.80254005]
 [-3.07697927  2.50687006 -0.69469937  0.80254005]]
x의 [0, 0]과 곱해진 W 원소들의 합
-3.0769792700509155

dw = 
[[ 3.10263293  3.10263293  3.10263293]
 [ 2.0555053   2.0555053   2.0555053 ]
 [ 1.66885811  1.66885811  1.66885811]
 [-0.25249778 -0.25249778 -0.25249778]]
W의 [0, 0]과 곱해진 x 원소들의 합
3.102632928972259

db = 
[2. 2. 2.]


### 이와 같이, 다양한 연산, 함수들에 대하여 forward와 backward를 정의할 수 있다

In [8]:
#활성화 함수들에 대한 순전파, 역전파


# if x < 0, y = 0
# otherwise, y = x
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

# y = 1/(1+exp(-x))
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    # ∂L/∂W = (∂L/∂y) * (1 - y) * y
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

class MSELoss:
    def __init__(self):
        self.loss = None
        self.error = None
    def forward(self, pred, target):
        self.error = pred-target
        self.loss = 0.5 * np.mean(self.error**2)
        return self.loss

    def backward(self, dout=1):
        mean_factor = 1 / self.error.size
        dpred = mean_factor * self.error
        return dpred

# ...등등. 자세한 것은 알아서 잘 찾아보도록 하자

### 신경망을 이용한 간단한 함수근사화
- 이번만 직접 작성해본 신경망을 이용하여 FrozenLake의 Q값을 근사화해보자
- 이후에서는 Tensorflow 2.x 버전을 이용하여 신경망을 설계 및 학습할 것이다.

### 신경망 정의

- 2층 신경망

In [9]:
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = MSELoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    # Mask: 어떤 부분만 Loss 계산할 것인지. 1이면 Loss 계산. 0이면 제외
    def loss(self, x, t, mask=1):
        y = self.predict(x)
        if np.size(mask) != 1:
            y = np.sum(y*mask, axis=1)
            t = np.sum(t*mask, axis=1)
        return self.lastLayer.forward(y, t)
        
    def gradient(self, x, t, mask=1):
        # forward
        self.loss(x, t, mask)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        # 마스크 연산 backpropagation
        dout = mask * dout[:, np.newaxis]
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### 신경망을 이용한 행동가치함수의 함수 근사화 (Deep Q Learning)

In [10]:
def find_and_show_q_table(network, env, agent):
    q_table = np.zeros((env.reward.shape[0], env.reward.shape[1], len(agent.action)))
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            for k in range(len(agent.action)): 
                q_table[i, j] = network.predict(np.array([[i,j]]))[0]
    print("Q Table (approximated by Neural Network)")
    show_q_table(np.round(q_table,2),env)
    print("Best actions Arrow")
    show_q_table_arrow(q_table,env)


np.random.seed(7777)


# 상태 (i, j)를 입력으로 받아 [상, 우, 하, 좌] 행동에 대한 Q값을 반환하는 신경망
network = TwoLayerNet(input_size=2, hidden_size=16, output_size=4, weight_init_std = 0.1)

# 1. 환경 초기화
env = Environment()

# 2. 에이전트 초기화
agent = Agent()

total_step = 2000
batch_size = 100
steps_for_print = 400
gamma = 0.9
learning_rate = 0.01
total_loss = 0

print("="*30)
print("step", 0)
find_and_show_q_table(network, env, agent)

for attmp in range(total_step):
    # 신경망의 입력 (state)
    x = np.zeros([batch_size, 2])
    # 신경망의 (우리가 원하는 바람직한) 정답
    target_y = np.zeros([batch_size, 4])
    # 어떤 출력을 보고자 하는 것인지 나타내는 mask.
    # mask[batch, k] = 1 : 현재 k란 행동을 수행한 경우를 학습하고 싶다
    mask = np.zeros([batch_size, 4])
    
    # (target_y를 구하기 위한) 임시값들
    observations = np.zeros([batch_size, 2])
    rewards = np.zeros([batch_size])
    dones = np.zeros([batch_size])
    
    #batch마다 데이터셋 (x, target_y, mask) 만들기
    for batch in range(batch_size):
        i, j = np.random.randint(0, 3, [2])
        while (i, j) == (2, 2):
            i, j = np.random.randint(0, 3, [2])
        x[batch] = (i, j)
        agent.set_pos([i,j])
        act = np.random.randint(len(agent.action))
        mask[batch, act] = 1
        observation, reward, done = env.move(agent, act)
        observations[batch] = observation
        rewards[batch] = reward
        dones[batch] = done
    next_pred = network.predict(observations)
    next_q = np.amax(next_pred, axis=1)
    target_y += (rewards + (1-dones) * gamma * next_q)[:, np.newaxis]
    
    #assert(False)
    
    # 기울기 계산
    grad = network.gradient(x, target_y, mask) 
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x, target_y, mask)
    total_loss += loss

    # 주기적으로 평균 Loss 출력
    if (attmp+1) % steps_for_print == 0:
        print("="*30)
        print("step", attmp+1, " loss:", total_loss/(steps_for_print*batch_size))
        total_loss = 0
        find_and_show_q_table(network, env, agent)
        
#4. 학습 후 차이(편차) 계산
devi_table = np.zeros((env.reward.shape[0], env.reward.shape[1],len(agent.action)))

for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        if (i, j) == (2, 2):
            continue
        for k in range(4):
            agent.set_pos([i,j])
            observation, reward, done = env.move(agent, k)
            if not done:
                next_q = np.max(network.predict(np.array([observation])))
                target = reward + gamma * next_q
            else:
                target = reward
            devi_table[i,j,k] = network.predict(np.array([[i,j]]))[0, k] - target
            
print()
print("Error Table")
show_q_table(np.round(devi_table,2), env)

step 0
Q Table (approximated by Neural Network)
+-----------------+-----------------+-----------------+
|      0.00       |     -0.03       |     -0.05       |
|  0.00      0.00 |  0.01      0.02 |  0.01      0.04 |
|      0.00       |      0.02       |      0.05       |
+-----------------+-----------------+-----------------+
|      0.03       |      0.03       |     -0.00       |
| -0.01     -0.02 | -0.03     -0.00 | -0.02      0.02 |
|     -0.02       |      0.01       |      0.04       |
+-----------------+-----------------+-----------------+
|      0.06       |      0.06       |      0.05       |
| -0.02     -0.05 | -0.05     -0.02 | -0.05     -0.00 |
|     -0.03       |     -0.01       |      0.02       |
+-----------------+-----------------+-----------------+
Best actions Arrow
+-----------------+-----------------+-----------------+
|        ↑       |                 |                 |
|      ←  →     |                 |                 |
|        ↓       |        ↓       |     